In [2]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from skills_matcher.model_script import get_train_data, minibatch_model, niter_model, get_skills, PATH
from skills_matcher.clean_data import clean_dictionary

data = pd.read_csv(PATH + "/data/dictionaries/new_dictionary.csv")
data = data[2000:]
data = clean_dictionary(data, sample = None)
df = pd.read_csv(PATH + '/data/cleaned_data.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df.description[2000]

' Ensuring customers are at the heart of everything we do, making sure our expertise and medicines are easily accessed, providing patient services, dispensing, and giving healthcare advice. Leading, motivating, training and developing your colleagues. Keeping the both the customer and the business safe and legal. Building working relationships with Primary Care Networks and GP surgeries Ensuring payment claims are accurate for anything covered by the NHS. Using our modern planning systems to ensure we have the right number of colleagues and pharmacists on duty to meet customer needs , A Pharmacy Degree A passion for leading and empowering a team. Membership of the General Pharmaceutical Council (GPhC) Hepatitis B immunisation (or be prepared to be immunised after starting) Pharmacists working for Tesco will have to complete a DBS check (organised by Tesco) prior to commencement of employment. , Annual payment of your GPhC fee. A structured training plan for Pharmacy Leadership, crafted

## Minibatch model

In [ ]:
%%time
train_data = get_train_data(data)
minibatch_model(train_data, 'minibatch_2', lang='en', pipe='ner', batch_size=1, drop=0.6)

In [ ]:
res_minibatch = get_skills('minibatch_2', df.description[400])

## N_iter model

In [ ]:
niter_model(train_data, 'n_iter', lang='en', pipe='ner', n_iter=1, drop=0.60)

In [ ]:
res_n_iter= get_skills('n_iter', df.description[340])

## Data Scientist

In [5]:
from IPython import get_ipython
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from skills_matcher.model_script import get_train_data, minibatch_model, niter_model, get_skills, PATH
from skills_matcher.clean_data import clean_dictionary

In [ ]:
data_2 = pd.read_csv(PATH + "/data/dictionaries/data_science_skills.csv")
data_2 = clean_dictionary(data_2, sample = None)

In [ ]:
ds_df = df.loc[df.job == 'data scientist']
ds_df = ds_df.reset_index().drop(['index', 'Unnamed: 0'], axis = 1)

In [ ]:
train_data = get_train_data(data_2)
train_data

In [ ]:
#minibatch_model(train_data, 'minibatch_4', lang='en', pipe='ner', batch_size=1, drop=0.1)

In [ ]:
#res_minibatch = get_skills('minibatch_4', ds_df.description[6])

In [6]:
df.description[1234]

" Long term opportunity of permanent Early Years Special Needs Teacher , SEN School , Full-time , Waltham Forest , Have experience as working as a Teacher in an SEN School , Have experience of Visual Impairment , Be able to travel to Waltham Forest , Available for full time and longer-term work , Have a DBS registered to the update service , Excellent daily rates paid weekly by our in-house Payroll team using the Pay As You Earn (PAYE) system. , Pension contributions (subject to a qualifying period). , Full compliance with AWR (Agency Workers' Regulations) , FREE training to help with your professional development , Generous refer a friend or colleague bonus scheme. , A dedicated consultant, who will provide ongoing support. "

In [ ]:
np.random.randint(100)

In [8]:
JD = df.description.drop_duplicates()
JD = JD.sample(200)

In [10]:
JD.to_csv('data_for_dom_dom.csv')

In [ ]:
JD = list(JD.values)

# Try Model with labeled data

In [18]:
import json
from IPython import get_ipython
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from skills_matcher.model_script import get_train_data, minibatch_model, niter_model, get_skills, PATH
from skills_matcher.clean_data import clean_dictionary

with open('annotation_data.json') as json_file:
    data = json.load(json_file)
    
from spacy import displacy
from spacy.training import Example
from spacy.training import biluo_tags_to_offsets
from pathlib import Path
from tqdm import tqdm
import numpy as np
import shutil
import spacy
import os

In [57]:
train_format = []
for i in range(len(data)):
    jd = data[i]['data']
    label = data[i]['label']
    label = [tuple(j) for j in label] 
    label = {'entities': label}
    row = (jd, label)
    train_format.append(row)

In [59]:
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(train_format): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("train.spacy") # save the docbin object

 86%|██████████████████████████████████████████████████████████████████████▊           | 63/73 [00:00<00:00, 144.34it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 143.93it/s]


In [43]:
#train_format[0]

label = data[0]['label']
label = {'entities': [[2, 3, 'SKILL'], [6, 7, 'SKILL']]}
jd = data[0]['data']
train_format = [jd, label]
train_format

[' Examples of previous work. Strong attention to detail and grammar. Great communication skills. Access to a PC / Laptop with Microsoft Word, & email. 2 references. , Fully remote - work for us from anywhere around the UK. Competitive rates on a schedule to suit your lifestyle.',
 {'entities': [[2, 3, 'SKILL'], [6, 7, 'SKILL']]}]

In [54]:
minibatch_model(train_format, 'minibatch_4', lang='en', pipe='ner', batch_size=1, drop=0.1)

0it [00:00, ?it/s]/home/macrodrigues/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text " High and low dusting, mopping, wiping and scrubbi..." with entities "[(63, 72, 'KNOWLEDGE'), (555, 582, 'SKILL'), (886,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
7it [00:00, 12.81it/s]/home/macrodrigues/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text " (restrictions apply in some instances due to the ..." with entities "[(794, 835, 'LEVEL'), (1463, 1477, 'KNOWLEDGE'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.

ValueError: [E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help

In [49]:
labels = ['KNOWLEDGE', 'MIN_EXP', 'LABEL', 'SKILL']

nlp = spacy.blank('en')  #create a blank model in english

# set pipeline
nlp.add_pipe('ner') # ner is an entity recognizer, detects labels.


for _, annotations in train_format:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# apply n_iter model
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(2):
        np.random.shuffle(train_format)
        losses = {}
        for text, annotations in tqdm(train_format):
            doc = nlp.make_doc(text)
            # Update the model
            print(annotations)
            example = Example.from_dict(doc, *annotations)
            # Update the model
            nlp.update([example],
                        sgd = optimizer,
                        losses=losses,
                        drop=drop)

ValueError: too many values to unpack (expected 2)